## Step 1. Initialization
In this step of the demonstration, I would just be importing the packages and defining a scene that I can operate on to visualize the Rotational operations I want to talk about

In [101]:
begin # Import necessary packages
    println("Loading packages...")
    using GLMakie, GeometryBasics, LinearAlgebra, Quaternions, FileIO
    import Quaternions: Quaternion as Quaternion
    println("Packages loaded successfully.")

    cd(@__DIR__) # Changing the current working directory from project root to the parent directory of this file so that pwd() and @__DIR__ match.
    # Import custom code and utility functions
    include("../src/State_and_Conversions.jl")
    include("../src/Rendering.jl")
    include("../src/Transformations.jl")
    include("../src/WindowManager.jl")
    include("../src/Recorder.jl")



    # Initialize the custom structs and utilities
    conversions = Conversions()
    renderer = Renderer()
    transformations = Transformations()
    windowmanager = WindowManager()

    println("Custom modules loaded successfully.")

    # Utility function to rotate the wing object
    function rotate_obj(obj::AbstractMesh, R::AbstractMatrix; about::Union{AbstractVector,Nothing}=nothing)
    obj_copy = deepcopy(obj)  # Create a copy of the object to avoid modifying the original
    # First translate the object to the origin
    if isnothing(about)
        # If no specific point is given, just rotate the object
        # Rotate positions
        for i in eachindex(obj.vertex_attributes[1])
            obj_copy.vertex_attributes[1][i] = Point3f(R * Vec3f(obj_copy.vertex_attributes[1][i]))
        end

        # Rotate normals if they exist
        if length(obj_copy.vertex_attributes) >= 3
            for i in eachindex(obj_copy.vertex_attributes[3])
                obj_copy.vertex_attributes[3][i] = normalize(Point3f(R * Vec3f(obj_copy.vertex_attributes[3][i])))
            end
        end

        return obj_copy
    else
        # If a specific point is given, translate the object to that point, rotate it, and then translate it back
        translation = -about
        obj_copy = translate_obj(obj_copy, translation)  # Translate to origin
        obj_copy = rotate_obj(obj_copy, R)  # Rotate around the origin
        obj_copy = translate_obj(obj_copy, -translation)  # Translate back to the original position

        return obj_copy
    end
end

function attach2State(obj::AbstractMesh, state_obs::Observable{State})
    return lift(st -> rotate_obj(obj, conversions.quat2rotmatrix(st.q)), state_obs)
end

    nothing
end

Loading packages...
Packages loaded successfully.
Custom modules loaded successfully.


In [122]:
# Create a Scene with standard basis arrows
scene = Scene(camera=cam3d!)
standard_basis_arrows = renderer.drawState!(scene)
display(scene)

GLMakie.Screen(...)

In [123]:
# Setup a State Observable initially set to standard state, along with a wing attatched to that state.

wingState = Observable(State()) # Initializes a Quaternion(1,0,0,0) by default
wing_obj = load(assetpath(joinpath(@__DIR__, "../Models/edited_wing.STL")))
wing_model = attach2State(wing_obj, wingState)
meshscatter!(scene, (0, 0, 0), marker=wing_model, markersize=0.0075, color=:gold, transparency=true, alpha=0.5)

vectors2show = [Point3(1.0, 0.0, 0.0), Point3(0.0, 1.0, 0.0), Point3(0.0, 0.0, 1.0)]
vectorcolors = [(1, 0, 0), (0, 1, 0), (0, 0, 1)] # red, green, blue
attached_vectors = renderer.attach2State(vectors2show, wingState) # Creates a vector of Observable Points that attaches the custom vectors to the observable state

# Draw the state with attached vectors
for i in eachindex(attached_vectors)
    meshscatter!(scene, (0, 0, 0), marker=lift(p -> Arrow(p).mesh, attached_vectors[i]), markersize=0.75, color=(vectorcolors[i]..., 0.25), transparency=true)
end


In [124]:
# Test the wingState Observable 
wingState[] = State(axisangle2quat([1.0, 1.0, 1.0], pi/4)) # -> Set state to a particular angle around an axis.

State([0.8047378541243649, 0.5058793634016806, -0.31061721752604554], [-0.31061721752604554, 0.8047378541243649, 0.5058793634016806], [0.5058793634016806, -0.31061721752604554, 0.8047378541243649], QuaternionF64(0.9238795325112867, 0.22094238269039457, 0.22094238269039457, 0.22094238269039457))

In [125]:
# Test Animation Visualization (You do not need to know the under the hood processing of this)
transformations.interpolate_states(wingState, State(), rate_function=t -> sin(t * pi / 2), time=2.0, n=100)

In [ ]:
q0 = Quaternion(1, 0, 0, 0)  # Identity quaternion
axis = [0, 0, 1]
angle = π/4  # 45 degrees in radians




## Step 2. Understanding Euler Angle Rotations

Euler angles represent rotations as three sequential rotations about coordinate axes. The two main conventions are:
- **Global (Extrinsic) XYZ**: Rotations about fixed global coordinate axes
- **Local (Intrinsic) XY'Z''**: Rotations about current local coordinate axes

### 2.1 Global XYZ Euler Angle Rotations

Global rotations are performed about the **fixed** global coordinate axes: **R = R_z(γ) · R_y(β) · R_x(α)**

In [114]:
# Initialize Global XYZ Rotation Scene
scene_global = Scene(camera=cam3d!)
windowmanager.closeall()
windowmanager.display(scene_global; names=["Global XYZ Rotations"])

renderer.drawState!(scene_global)

# Setup state and wing model
wingstate_global = Observable(State())
wing_obj = load(assetpath(joinpath(@__DIR__, "../Models/edited_wing.STL")))
wing_model = attach2State(wing_obj, wingstate_global)
meshscatter!(scene_global, (0, 0, 0), marker=wing_model, markersize=0.0075, color=:gold, transparency=true, alpha=0.5)

# Setup vectors
vectors2show = [Point3(1.0, 0.0, 0.0), Point3(0.0, 1.0, 0.0), Point3(0.0, 0.0, 1.0)]
vectorcolors = [(1, 0, 0), (0, 1, 0), (0, 0, 1)]
attached_vectors_global = renderer.attach2State(vectors2show, wingstate_global)

for i in eachindex(attached_vectors_global)
    meshscatter!(scene_global, (0, 0, 0), marker=lift(p -> Arrow(p).mesh, attached_vectors_global[i]), markersize=0.75, color=(vectorcolors[i]..., 0.5), transparency=true)
end

In [115]:
# Apply Global XYZ Rotations
α, β, γ = 0, 0, π/6  # 0°, 0°, 30°

rotation_x = conversions.axisangle2quat([1, 0, 0], α)
rotation_y = conversions.axisangle2quat([0, 1, 0], β)
rotation_z = conversions.axisangle2quat([0, 0, 1], γ)

# Global rotations: about fixed axes (opState=false)
transformations.combine_rotation_interpolations(wingstate_global,
    (rotation_x, false),  # X about global axis
    (rotation_y, false),  # Y about global axis  
    (rotation_z, false);  # Z about global axis
    n=100, time=2.0, rate_function=t -> sin(t * pi / 2)
)



In [117]:
p = Point3(1.0, 0.0, 0.0)
p_rot = conversions.rotate_vector(p, wingstate_global[].q)  
# Rotate point p by rotation_x

print(p_rot)
meshscatter!(scene_global, Point3(0.0), marker=Arrow(p_rot).mesh, markersize=1.0, color=(:purple, 0.75), transparency=true, )

[0.8660254037844387, 0.5, 0.0]

MeshScatter{Tuple{Vector{Point{3, Float64}}}}

### 2.2 Local XY'Z'' Euler Angle Rotations

Local rotations are performed about the **current** local coordinate axes: **R = R_x(α) · R_y'(β) · R_z''(γ)**

In [119]:
# Initialize Local XY'Z'' Rotation Scene
scene_local = Scene(camera=cam3d!)
windowmanager.closeall()
windowmanager.display(scene_local; names=["Local XY'Z'' Rotations"])

renderer.drawState!(scene_local)

# Setup state and wing model
wingstate_local = Observable(State())
wing_model_local = attach2State(wing_obj, wingstate_local)
meshscatter!(scene_local, (0, 0, 0), marker=wing_model_local, markersize=0.0075, color=:gold, transparency=true, alpha=0.5)

# Setup vectors
attached_vectors_local = renderer.attach2State(vectors2show, wingstate_local)

for i in eachindex(attached_vectors_local)
    meshscatter!(scene_local, (0, 0, 0), marker=lift(p -> Arrow(p).mesh, attached_vectors_local[i]), markersize=0.75, color=(vectorcolors[i]..., 0.5), transparency=true)
end

In [120]:
# Apply Local XY'Z'' Rotations
α, β, γ = -π/4, π/6, 0  # -45°, 30°, 0°

rotation_x = conversions.axisangle2quat([1, 0, 0], α)
rotation_y = conversions.axisangle2quat([0, 1, 0], β)
rotation_z = conversions.axisangle2quat([0, 0, 1], γ)

# Local rotations: about current local axes (opState=true)
transformations.combine_rotation_interpolations(wingstate_local,
    (rotation_x, true),   # X about current local axis
    (rotation_y, true),   # Y about current local axis
    (rotation_z, true);   # Z about current local axis
    n=100, time=2.0, rate_function=t -> sin(t * pi / 2)
)

In [ ]:
p = Point3(1.0, 0.0, 0.0)
p_rot = conversions.rotate_vector(p, wingstate_local[].q)  
# Rotate point p by rotation_x

print(p_rot)
meshscatter!(scene_global, Point3(0.0), marker=Arrow(p_rot).mesh, markersize=1.0, color=(:purple, 0.75), transparency=true, )

In [121]:
wingstate_local[].q

QuaternionF64(0.8923991008325227, -0.36964381061438595, 0.2391176183943344, -0.0990457605412876)

### 2.3 Side-by-Side Comparison: Global vs Local

Let's compare both methods side-by-side using the same angles.

In [47]:
# Initialize Side-by-Side Comparison
scene_global_comp = Scene(camera=cam3d!)
scene_local_comp = Scene(camera=cam3d!)

windowmanager.closeall()
windowmanager.display(scene_global_comp, scene_local_comp; names=["Global XYZ", "Local XY'Z''"])

# Setup both scenes
for scene in [scene_global_comp, scene_local_comp]
    renderer.drawState!(scene)
end

# Global scene setup
wingstate_global_comp = Observable(State())
wing_model_global_comp = attach2State(wing_obj, wingstate_global_comp)
meshscatter!(scene_global_comp, (0, 0, 0), marker=wing_model_global_comp, markersize=0.0075, color=:gold, transparency=true, alpha=0.5)
attached_vectors_global_comp = renderer.attach2State(vectors2show, wingstate_global_comp)
for i in eachindex(attached_vectors_global_comp)
    meshscatter!(scene_global_comp, (0, 0, 0), marker=lift(p -> Arrow(p).mesh, attached_vectors_global_comp[i]), markersize=0.75, color=(vectorcolors[i]..., 0.5), transparency=true)
end

# Local scene setup
wingstate_local_comp = Observable(State())
wing_model_local_comp = attach2State(wing_obj, wingstate_local_comp)
meshscatter!(scene_local_comp, (0, 0, 0), marker=wing_model_local_comp, markersize=0.0075, color=:gold, transparency=true, alpha=0.5)
attached_vectors_local_comp = renderer.attach2State(vectors2show, wingstate_local_comp)
for i in eachindex(attached_vectors_local_comp)
    meshscatter!(scene_local_comp, (0, 0, 0), marker=lift(p -> Arrow(p).mesh, attached_vectors_local_comp[i]), markersize=0.75, color=(vectorcolors[i]..., 0.5), transparency=true)
end

In [49]:
# Apply Rotations Side-by-Side
α, β, γ = π/4, π/6, π/3  # 45°, 30°, 60°

rotation_x = conversions.axisangle2quat([1, 0, 0], α)
rotation_y = conversions.axisangle2quat([0, 1, 0], β)
rotation_z = conversions.axisangle2quat([0, 0, 1], γ)

# Run both simultaneously - same total time (6 seconds)
@async transformations.combine_rotation_interpolations(wingstate_global_comp,
    (rotation_x, false), (rotation_y, false), (rotation_z, false);
    n=100, time=2.0, rate_function=t -> sin(t * pi / 2)
)

@async transformations.combine_rotation_interpolations(wingstate_local_comp,
    (rotation_x, true), (rotation_y, true), (rotation_z, true);
    n=100, time=2.0, rate_function=t -> sin(t * pi / 2)
)

Task (runnable, started) @0x000001dac3b87840

### 2.4 The Non-Linearity Problem

**Problem**: Euler angles don't add simply! 
Rotating by (α,β,γ) then by (a,b,c) ≠ rotating by (α+a, β+b, γ+c)

Let's demonstrate this fundamental limitation:

In [ ]:
# Initialize Non-Linearity Demonstration
scene_wrong = Scene(camera=cam3d!)
scene_correct = Scene(camera=cam3d!)

windowmanager.closeall()
windowmanager.display(scene_wrong, scene_correct; names=["Simple Addition", "Sequential Application"])

# Setup both scenes
for scene in [scene_wrong, scene_correct]
    renderer.drawState!(scene)
end

# Wrong method scene
wingstate_wrong = Observable(State())
wing_model_wrong = attach2State(wing_obj, wingstate_wrong)
meshscatter!(scene_wrong, (0, 0, 0), marker=wing_model_wrong, markersize=0.0075, color=:gold, transparency=true, alpha=0.5)
attached_vectors_wrong = renderer.attach2State(vectors2show, wingstate_wrong)
for i in eachindex(attached_vectors_wrong)
    meshscatter!(scene_wrong, (0, 0, 0), marker=lift(p -> Arrow(p).mesh, attached_vectors_wrong[i]), markersize=0.75, color=(vectorcolors[i]..., 0.5), transparency=true)
end

# Correct method scene
wingstate_correct = Observable(State())
wing_model_correct = attach2State(wing_obj, wingstate_correct)
meshscatter!(scene_correct, (0, 0, 0), marker=wing_model_correct, markersize=0.0075, color=:gold, transparency=true, alpha=0.5)
attached_vectors_correct = renderer.attach2State(vectors2show, wingstate_correct)
for i in eachindex(attached_vectors_correct)
    meshscatter!(scene_correct, (0, 0, 0), marker=lift(p -> Arrow(p).mesh, attached_vectors_correct[i]), markersize=0.75, color=(vectorcolors[i]..., 0.5), transparency=true)
end

In [54]:
# Demonstrate Wrong vs Correct Rotation Composition
α₁, β₁, γ₁ = π/6, π/4, π/3   # First: 30°, 45°, 60°
a, b, c = π/10, π/10, π/10      # Second: 45°, 30°, 45°
# a,b,c=0.0,0.0,0.0

# WRONG: Simple addition of angles
α_wrong = α₁ + a  # 75°
β_wrong = β₁ + b  # 75°
γ_wrong = γ₁ + c  # 105°

rot_x_wrong = conversions.axisangle2quat([1, 0, 0], α_wrong)
rot_y_wrong = conversions.axisangle2quat([0, 1, 0], β_wrong)
rot_z_wrong = conversions.axisangle2quat([0, 0, 1], γ_wrong)

# CORRECT: Sequential application
rot1_x = conversions.axisangle2quat([1, 0, 0], α₁)
rot1_y = conversions.axisangle2quat([0, 1, 0], β₁)
rot1_z = conversions.axisangle2quat([0, 0, 1], γ₁)
rot2_x = conversions.axisangle2quat([1, 0, 0], a)
rot2_y = conversions.axisangle2quat([0, 1, 0], b)
rot2_z = conversions.axisangle2quat([0, 0, 1], c)

# Execute both - same total time (12 seconds)
@async transformations.combine_rotation_interpolations(wingstate_wrong,
    (rot_x_wrong, false), (rot_y_wrong, false), (rot_z_wrong, false);
    n=100, time=2.0, rate_function=t -> sin(t * pi / 2)
)

@async transformations.combine_rotation_interpolations(wingstate_correct,
    (rot1_x, false), (rot1_y, false), (rot1_z, false),
    (rot2_x, false), (rot2_y, false), (rot2_z, false);
    n=100, time=2.0, rate_function=t -> sin(t * pi / 2)
)

Task (runnable, started) @0x000001dac3b849c0

In [58]:
# Superimposed Comparison
scene_superimposed = Scene(camera=cam3d!)
windowmanager.closeall()
windowmanager.display(scene_superimposed; names=["Superimposed: Red=Simple Addition, Green=Combination"])

renderer.drawState!(scene_superimposed)

# Red vectors for wrong method
wingstate_super_wrong = Observable(State())
wing_model_super_wrong = attach2State(wing_obj, wingstate_super_wrong)
meshscatter!(scene_superimposed, (0, 0, 0), marker=wing_model_super_wrong, markersize=0.0075, color=:red, transparency=true, alpha=0.3)
attached_vectors_super_wrong = renderer.attach2State(vectors2show, wingstate_super_wrong)
for i in eachindex(attached_vectors_super_wrong)
    meshscatter!(scene_superimposed, (0, 0, 0), marker=lift(p -> Arrow(p).mesh, attached_vectors_super_wrong[i]), markersize=0.75, color=(1.0, 0.2, 0.2, 0.6), transparency=true)
end

# Green vectors for correct method
wingstate_super_correct = Observable(State())
wing_model_super_correct = attach2State(wing_obj, wingstate_super_correct)
meshscatter!(scene_superimposed, (0, 0, 0), marker=wing_model_super_correct, markersize=0.0075, color=:green, transparency=true, alpha=0.3)
attached_vectors_super_correct = renderer.attach2State(vectors2show, wingstate_super_correct)
for i in eachindex(attached_vectors_super_correct)
    meshscatter!(scene_superimposed, (0, 0, 0), marker=lift(p -> Arrow(p).mesh, attached_vectors_super_correct[i]), markersize=0.75, color=(0.2, 1.0, 0.2, 0.6), transparency=true)
end

sleep(5.0)

# Execute superimposed demonstration - same total time
@async transformations.combine_rotation_interpolations(wingstate_super_wrong,
    (rot_x_wrong, false), (rot_y_wrong, false), (rot_z_wrong, false);
    n=100, time=2.0, rate_function=t -> sin(t * pi / 2)
)

@async transformations.combine_rotation_interpolations(wingstate_super_correct,
    (rot1_x, false), (rot1_y, false), (rot1_z, false),
    (rot2_x, false), (rot2_y, false), (rot2_z, false);
    n=100, time=2.0, rate_function=t -> sin(t * pi / 2)
)

Task (runnable, started) @0x000001dac3b85180

### 2.5 The Reset Workaround

Since Euler angles don't compose simply, a common workaround is to **reset to zero** before applying new rotations.

In [69]:
# Initialize Reset Demonstration
scene_reset = Scene(camera=cam3d!)
windowmanager.closeall()
windowmanager.display(scene_reset; names=["Reset-Then-Apply Method"])

renderer.drawState!(scene_reset)

wingstate_reset = Observable(State())
wing_model_reset = attach2State(wing_obj, wingstate_reset)
meshscatter!(scene_reset, (0, 0, 0), marker=wing_model_reset, markersize=0.0075, color=:gold, transparency=true, alpha=0.5)
attached_vectors_reset = renderer.attach2State(vectors2show, wingstate_reset)
for i in eachindex(attached_vectors_reset)
    meshscatter!(scene_reset, (0, 0, 0), marker=lift(p -> Arrow(p).mesh, attached_vectors_reset[i]), markersize=0.75, color=(vectorcolors[i]..., 0.5), transparency=true)
end


# Reset Demonstration
α₁, β₁, γ₁ = π/6, π/4, π/3   # First: 30°, 45°, 60°
a, b, c = π/4, π/6, π/4      # Second: 45°, 30°, 45°

# Target: combined result (α₁+a, β₁+b, γ₁+c)
α_target = α₁ + a
β_target = β₁ + b  
γ_target = γ₁ + c

rot1_x = conversions.axisangle2quat([1, 0, 0], α₁)
rot1_y = conversions.axisangle2quat([0, 1, 0], β₁)
rot1_z = conversions.axisangle2quat([0, 0, 1], γ₁)

rot_target_x = conversions.axisangle2quat([1, 0, 0], α_target)
rot_target_y = conversions.axisangle2quat([0, 1, 0], β_target)
rot_target_z = conversions.axisangle2quat([0, 0, 1], γ_target)


# Apply first rotation
transformations.combine_rotation_interpolations(wingstate_reset,
    (rot1_x, false), (rot1_y, false), (rot1_z, false);
    n=100, time=1.0, rate_function=t -> sin(t * pi / 2)
)

sleep(7.0)

# --- Add ghost for the target state ---
target_quat = rot_target_z * rot_target_y * rot_target_x
ghost_vectors = renderer.attach2State(vectors2show, Observable(State(target_quat)))
for i in eachindex(ghost_vectors)
    meshscatter!(scene_reset, (0, 0, 0), marker=lift(p -> Arrow(p).mesh, ghost_vectors[i]),
        markersize=0.75, color=(custom_vector_colors[i]..., 0.2), transparency=true)
end
ghost_wing = attach2State(wing_obj, Observable(State(target_quat)))
meshscatter!(scene_reset, (0, 0, 0), marker=ghost_wing, markersize=0.0075, color=:gold, transparency=true, alpha=0.20)

# --- End ghost ---

sleep(3.0)
# Reset to zero (discontinuous!)
transformations.interpolate_states(wingstate_reset, State(); n=50, time=1.0, rate_function=t -> sin(t * pi / 2))


sleep(5.0)

# Apply target rotation
transformations.combine_rotation_interpolations(wingstate_reset,
    (rot_target_x, false), (rot_target_y, false), (rot_target_z, false);
    n=100, time=2.0, rate_function=t -> sin(t * pi / 2)
)

#### The Problem

The fundamental issue with Euler angle rotations is that they are **order-dependent** and **nonlinear**. In the XYZ convention, a rotation is defined as a sequence: first about the global X-axis by $\alpha$, then about the global Y-axis by $\beta$, and finally about the global Z-axis by $\gamma$.

Suppose you perform a rotation by $(\alpha, \beta, \gamma)$ (i.e., $\alpha$ about X, $\beta$ about Y, $\gamma$ about Z), and then perform another rotation by $(a, b, c)$ (i.e., $a$ about X, $b$ about Y, $c$ about Z). This sequence is **not** equivalent to a single rotation by $(\alpha+a,\, \beta+b,\, \gamma+c)$.

Mathematically, if we denote the composite rotation as:
$$
\text{Composite}(X, \alpha;\ Y, \beta;\ Z, \gamma)
$$
then applying two such sequences in order gives:
$$
\text{Composite}(X, \alpha;\ Y, \beta;\ Z, \gamma;\ X, a;\ Y, b;\ Z, c) \neq \text{Composite}(X, \alpha+a;\ Y, \beta+b;\ Z, \gamma+c)
$$

This is because the axes themselves are rotated after each step, so subsequent rotations are about **different axes** than the originals. The composition of rotations is **not additive** in Euler angles.

In summary:  
- **Euler angle rotations are not linear or additive.**
- **The order and grouping of rotations fundamentally change the result.**
- **Matrix and quaternion multiplication compose rotations correctly, but Euler angles do not.**

(More formally, the mapping from Euler angles to rotation matrices is nonlinear and non-commutative, so the sum of angles does not correspond to the product of rotations.)

## Step 3. Quaternions: The Superior Solution

### 3.1 Quaternion Incremental Rotations

With quaternions, incremental rotations are simple: **q\* = q₁ · q₀⁻¹**

Suppose you have an initial orientation represented by quaternion **q₀**, and you want to reach a new orientation **q₁**. The question is:  
**What quaternion q\*** should you apply to **q₀** so that you arrive at **q₁**?

Mathematically, we want:
  
$$
q^* \cdot q_0 = q_1
$$

To solve for **q\***, multiply both sides by the inverse of **q₀**:

$$
q^* = q_1 \cdot q_0^{-1}
$$

For unit quaternions (which represent rotations), the inverse is just the conjugate:

$$
q_0^{-1} = \overline{q_0}
$$

So the incremental quaternion is:

$$
q^* = q_1 \cdot \overline{q_0}
$$

This means you can smoothly interpolate or "step" from **q₀** to **q₁** by applying **q\***, without any need to reset or decompose into Euler angles. This property is one of the main reasons quaternions are so powerful for representing and composing 3D rotations.

In [73]:
# Initialize Quaternion Incremental Rotation Demonstration
scene_quat_inc = Scene(camera=cam3d!)
windowmanager.closeall()
windowmanager.display(scene_quat_inc; names=["Quaternion Incremental Rotation"])

renderer.drawState!(scene_quat_inc)

wingstate_quat_inc = Observable(State())
wing_model_quat_inc = attach2State(wing_obj, wingstate_quat_inc)
meshscatter!(scene_quat_inc, (0, 0, 0), marker=wing_model_quat_inc, markersize=0.0075, color=:gold, transparency=true, alpha=0.5)
attached_vectors_quat_inc = renderer.attach2State(vectors2show, wingstate_quat_inc)
for i in eachindex(attached_vectors_quat_inc)
    meshscatter!(scene_quat_inc, (0, 0, 0), marker=lift(p -> Arrow(p).mesh, attached_vectors_quat_inc[i]), markersize=0.75, color=(vectorcolors[i]..., 0.5), transparency=true)
end

# Quaternion Incremental Rotation Demonstration
α₁, β₁, γ₁ = π/6, π/4, π/3   # First: 30°, 45°, 60°
a, b, c = π/4, π/6, π/4      # Second: 45°, 30°, 45°

# First rotation
rot1_x = conversions.axisangle2quat([1, 0, 0], α₁)
rot1_y = conversions.axisangle2quat([0, 1, 0], β₁)
rot1_z = conversions.axisangle2quat([0, 0, 1], γ₁)
q1 = rot1_z * rot1_y * rot1_x

# Second rotation
rot2_x = conversions.axisangle2quat([1, 0, 0], a)
rot2_y = conversions.axisangle2quat([0, 1, 0], b)
rot2_z = conversions.axisangle2quat([0, 0, 1], c)
q2 = rot2_z * rot2_y * rot2_x

# Final combined quaternion
q_final = q2 * q1

sleep(2.0)

# Apply first rotation
transformations.interpolate_states(wingstate_quat_inc, State(q1); n=100, time=2.0, rate_function=t -> sin(t * pi / 2))

sleep(2.5)

# --- Add ghost for the target state ---
ghost_vectors = renderer.attach2State(vectors2show, Observable(State(q_final)))
for i in eachindex(ghost_vectors)
    meshscatter!(scene_quat_inc, (0, 0, 0), marker=lift(p -> Arrow(p).mesh, ghost_vectors[i]),
        markersize=0.75, color=(custom_vector_colors[i]..., 0.2), transparency=true)
end
ghost_wing = attach2State(wing_obj, Observable(State(q_final)))
meshscatter!(scene_quat_inc, (0, 0, 0), marker=ghost_wing, markersize=0.0075, color=:gold, transparency=true, alpha=0.20)
# --- End ghost ---

sleep(2.0)

# Calculate incremental quaternion: q* = q_final * inv(q_current)
q_current = wingstate_quat_inc[].q
q_star = q_final * inv(q_current)

println("Current quaternion: $q_current")
println("Target quaternion: $q_final") 
println("Incremental quaternion q*: $q_star")

# Apply incremental rotation (NO RESET!)
transformations.interpolate_states(wingstate_quat_inc, State(q_final); n=100, time=2.0, rate_function=t -> sin(t * pi / 2))

Current quaternion: QuaternionF64(0.8223631719059993, 0.022260026714733844, 0.4396797395409095, 0.3604234056503559)
Target quaternion: QuaternionF64(0.4541846553482916, 0.24547479893520385, 0.5926285888101774, 0.6182635179749265)
Incremental quaternion q*: QuaternionF64(0.8623724356957945, 0.25, 0.3623724356957945, 0.24999999999999994)


### 3.2 Advanced Euler Angle Factorization Solution

Using quaternions and matrix decomposition, we can achieve incremental Euler rotations without reset.

Computing Euler angles from a rotation matrix
Gregory G. Slabaugh
https://eecs.qmul.ac.uk/~gslabaugh/publications/euler.pdf

In [97]:
# Define the Euler angle extraction function
function euler_angles_xyz(R::AbstractMatrix{<:Real})
    a11, a12, a13 = R[1,1], R[1,2], R[1,3]
    a21, a22, a23 = R[2,1], R[2,2], R[2,3]
    a31, a32, a33 = R[3,1], R[3,2], R[3,3]

    if abs(a13) != 1
        θ = asin(a13)
        φ = atan(-a23, a33)
        ψ = atan(-a12, a11)
    elseif a13 == 1
        θ = π/2
        φ = atan(a21, a22)
        ψ = 0  # Arbitrary; we pick 0
    elseif a13 == -1
        θ = -π/2
        φ = atan(-a21, -a22)
        ψ = 0  # Arbitrary; we pick 0
    end

    return φ, θ, ψ  # Local XYZ angles: φ (X/roll), θ (Y/pitch), ψ (Z/yaw)
end

# Initialize Euler Incremental Rotation Demonstration (Extension for Advanced Euler Angle Factorization)
scene_euler_inc = Scene(camera=cam3d!)
windowmanager.closeall()
windowmanager.display(scene_quat_inc, scene_euler_inc; names=["Quaternion Incremental Rotation", "Euler Angle Factorization Incremental (local XYZ)"])

renderer.drawState!(scene_euler_inc)

wingstate_euler_inc = Observable(State())
wing_model_euler_inc = attach2State(wing_obj, wingstate_euler_inc)
meshscatter!(scene_euler_inc, (0, 0, 0), marker=wing_model_euler_inc, markersize=0.0075, color=:gold, transparency=true, alpha=0.5)
attached_vectors_euler_inc = renderer.attach2State(vectors2show, wingstate_euler_inc)
for i in eachindex(attached_vectors_euler_inc)
    meshscatter!(scene_euler_inc, (0, 0, 0), marker=lift(p -> Arrow(p).mesh, attached_vectors_euler_inc[i]), markersize=0.75, color=(vectorcolors[i]..., 0.5), transparency=true)
end

# Euler Incremental Rotation Demonstration using Factorization
α₁, β₁, γ₁ = 0*π/6, 0*π/4, 0*π/3   # First: 30°, 45°, 60°
a, b, c = π/4, π/6, π/4      # Second: 45°, 30°, 45°

# First rotation quaternions (for composition reference)
rot1_x = conversions.axisangle2quat([1, 0, 0], α₁)
rot1_y = conversions.axisangle2quat([0, 1, 0], β₁)
rot1_z = conversions.axisangle2quat([0, 0, 1], γ₁)
q1 = rot1_x * rot1_y * rot1_z

# Second rotation quaternions
rot2_x = conversions.axisangle2quat([1, 0, 0], a)
rot2_y = conversions.axisangle2quat([0, 1, 0], b)
rot2_z = conversions.axisangle2quat([0, 0, 1], c)
q2 = rot2_x * rot2_y * rot2_z

# Final combined quaternion
q_final = q2*q1

sleep(2.0)

# Apply first rotation using sequential local XYZ Euler interpolations (intrinsic)
transformations.combine_rotation_interpolations(wingstate_euler_inc, 
    (rot1_x, true),  # Rotate around x-axis in local frame
    (rot1_y, true),  # Rotate around y-axis in local frame
    (rot1_z, true);  # Rotate around z-axis in local frame
    n=100, time=2.0, rate_function=t -> sin(t * pi / 2)
)

sleep(2.5)

# --- Add ghost for the target state ---
ghost_vectors = renderer.attach2State(vectors2show, Observable(State(q_final)))
for i in eachindex(ghost_vectors)
    meshscatter!(scene_euler_inc, (0, 0, 0), marker=lift(p -> Arrow(p).mesh, ghost_vectors[i]),
        markersize=0.75, color=(custom_vector_colors[i]..., 0.2), transparency=true)
end
ghost_wing = attach2State(wing_obj, Observable(State(q_final)))
meshscatter!(scene_euler_inc, (0, 0, 0), marker=ghost_wing, markersize=0.0075, color=:gold, transparency=true, alpha=0.20)
# --- End ghost ---

sleep(2.0)

# Advanced Euler Angle Factorization for Incremental Rotation (without reset)
q_current = wingstate_euler_inc[].q
q_star = q_final * inv(q_current)

println("Current quaternion: $q_current")
println("Target quaternion: $q_final") 
println("Incremental quaternion q*: $q_star")

# Decompose the incremental quaternion into local XYZ Euler angles
R_star = conversions.quat2rotmatrix(q_star)
delta_φ, delta_θ, delta_ψ = euler_angles_xyz(R_star)

println("Incremental Euler angles (factorized): φ=$delta_φ (X), θ=$delta_θ (Y), ψ=$delta_ψ (Z)")

# Apply the factorized incremental Euler rotations sequentially in local frame (NO RESET!)
transformations.combine_rotation_interpolations(wingstate_euler_inc, 
    (conversions.axisangle2quat([1, 0, 0], delta_φ), true),   # Incremental X rotation
    (conversions.axisangle2quat([0, 1, 0], delta_θ), true),   # Incremental Y rotation
    (conversions.axisangle2quat([0, 0, 1], delta_ψ), true);   # Incremental Z rotation
    n=100, time=2.0, rate_function=t -> sin(t * pi / 2)
)

# Optional: Verify final state matches target
sleep(2.5)
println("Final Euler state approx q_final? ", isapprox(wingstate_euler_inc[].q, q_final, atol=1e-10))

Current quaternion: QuaternionF64(1.0, 0.0, 0.0, 0.0)
Target quaternion: QuaternionF64(0.7865660924854931, 0.4330127018922193, 0.07945931129894551, 0.43301270189221935)
Incremental quaternion q*: QuaternionF64(0.7865660924854931, 0.4330127018922193, 0.07945931129894551, 0.43301270189221935)
Incremental Euler angles (factorized): φ=0.7853981633974483 (X), θ=0.5235987755982988 (Y), ψ=0.7853981633974485 (Z)
Final Euler state approx q_final? true


## Conclusion: Why Quaternions?

### Problems with Euler Angles:
1. **Non-linearity**: (α+a, β+b, γ+c) ≠ proper rotation composition
2. **Gimbal Lock**: Loss of degrees of freedom at certain orientations
3. **Discontinuous interpolation**: Poor for animation and control
4. **Reset requirement**: Often need to reset to zero for new rotations

### Advantages of Quaternions:
1. **Simple composition**: q₂ · q₁ naturally represents sequential rotations
2. **Smooth interpolation**: SLERP provides optimal rotation paths  
3. **No singularities**: No gimbal lock issues
4. **Incremental rotations**: q* = q₁ · q₀⁻¹ makes incremental changes trivial
5. **Efficient computation**: Fast multiplication and conjugation

**For any application requiring smooth rotation control, quaternions are superior!**